In [3]:
import os
os.environ["PYSPARK_PYTHON"] = "/Users/seanwu/opt/anaconda3/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/Users/seanwu/opt/anaconda3/bin/python"
from pyspark.sql import SparkSession
import json
import tldextract # extracts TLDs


spark = SparkSession \
    .builder \
    .appName("Python Spark basic example") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.3, graphframes:graphframes:0.3.0-spark2.0-s_2.11") \
    .getOrCreate()
#.config("spark.jars.packages", "graphframes:graphframes:0.3.0-spark2.0-s_2.11") \

Exception: Java gateway process exited before sending its port number

In [19]:
def TLDs():
    pass
def get_json(line):
    try:
        line = line.lower()
        return json.loads(line)
    except:
        pass

In [20]:
def get_links(x):
    try:
        links = []
        for dics in x['envelope']["payload-metadata"]\
            ["http-response-metadata"]\
            ["html-metadata"]\
            ["links"]:
            if dics["url"]:
                links.append(tldextract.extract(dics["url"]).join('.'))
        return links
    except:
        pass

In [21]:
def get_target_TLD(x):
    try:
        sub, do, suffix = tldextract.extract(x['envelope']['warc-header-metadata']['warc-target-uri'])
        return sub+'.'+do+'.'+suffix
    except:
        return None

### Read in text file

In [30]:
distText = spark.sparkContext.textFile('/Users/seanwu/Desktop/learn/spark/February 2016/crawl-data/CC-MAIN-2016-07/segments/1454702039825.90/wat/*.gz')

In [31]:
# distText = distText.map(lambda x: get_json(x)).filter(lambda x: x != None)

In [32]:
testFile = distText.map(lambda x: get_json(x)).filter(lambda x: x != None)\
    .map(lambda x:get_target_TLD(x)).filter(lambda x: x!= None)

In [33]:
testFile.take(10)

['0.r.msn.com',
 '0.r.msn.com',
 '0.r.msn.com',
 '1.bp.blogspot.com',
 '1.bp.blogspot.com',
 '1.bp.blogspot.com',
 '1015theriver.iheart.com',
 '1015theriver.iheart.com',
 '1015theriver.iheart.com',
 '1019ampradio.cbslocal.com']

In [11]:
from functools import reduce
from pyspark.sql.functions import col, lit, when
from graphframes import *

In [12]:
vertices = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 36),
  ("g", "Gabby", 60)], ["id", "name", "age"])

In [13]:
edges = spark.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
  ("f", "c", "follow"),
  ("e", "f", "follow"),
  ("e", "d", "friend"),
  ("d", "a", "friend"),
  ("a", "e", "friend")
], ["src", "dst", "relationship"])

### Create GraphFrame

In [14]:
g = GraphFrame(vertices, edges)

In [ ]:
g

### find youngest age

In [ ]:
youngest = g.vertices.groupBy().min("age")

### Do LPA

In [ ]:
result = g.labelPropagation(maxIter=5)

In [ ]:
result.collect()

In [ ]:
from pyspark.sql import *

### TODO: Use .collect() to turn pyspark.rdd.PipelinedRDD to a normal RDD and save

### TODO: save dataframes into table